In [90]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import numpy
CC = ComplexField(32)
pi = CC.pi()
I = CC.gen()

def generate_hnp_instance(leak_size, number_of_signatures, prime, key):
    #h = c*key - k
    hnp_instance = []

    bit_length = floor(log(prime, 2))+1
    bound = 2**(bit_length-leak_size-1) #-1 protoze centerujeme k
    
    for j in range(number_of_signatures):
        c = randrange(1, prime)
        k = randrange(-round(bound), floor(bound))
        h = (k - c*key) % prime
        hnp_instance.append([c, h])
    return hnp_instance

def generate_hnp_instance_bounded(leak_size, number_of_signatures, prime, key, bound_exponent):
    #h = k - c*key 
    hnp_instance = []

    bit_length = floor(log(prime, 2))+1
    bound = 2**(bit_length-leak_size-1) #-1 protoze centerujeme k
    
    for j in range(number_of_signatures):
        c = randrange(1, 2**bound_exponent % prime)
        k = randrange(-round(bound), floor(bound))
        h = (k - c*key) % prime
        hnp_instance.append([c, h])
    return hnp_instance


def sort_and_difference(hnp_instance, prime, number_of_iterations, bound_exponent):
    new_instance = hnp_instance
    for i in range(number_of_iterations):
        new_instance.sort(key=lambda x:x[0])
        helper_list = []
        for j in range(len(new_instance)-1):
            if i < len(new_instance)-2 | (new_instance[j+1][0]-new_instance[j][0]) < 2**bound_exponent:
                helper_list.append(((new_instance[j+1][0]-new_instance[j][0]), (new_instance[j+1][1]-new_instance[j][1])% prime))
        new_instance = helper_list.copy()
    return new_instance

def bleichenbacher(hnp_instance, prime, fft_bound_exponent):
    z = [0] * (2**fft_bound_exponent + 1)
    for i in range(len(hnp_instance)-1):
        z[hnp_instance[i][0]] += exp(2*pi*I* hnp_instance[i][1]/prime)
    coordinates = numpy.fft.fft(z)
    magnitudes = numpy.abs(coordinates)
    
    return numpy.argmax(magnitudes)*prime/2**(fft_bound_exponent)

def bleichenbacher_bounded_key(hnp_instance, prime, fft_bound_exponent, key_bound_exponent):
    z = [0] * (2**fft_bound_exponent + 1)
    for i in range(len(hnp_instance)-1):
        z[round(hnp_instance[i][0]*2**key_bound_exponent/prime)] += exp(2*pi*I* hnp_instance[i][1]/prime)
    coordinates = numpy.fft.fft(z)
    magnitudes = numpy.abs(coordinates)
    
    return numpy.argmax(magnitudes)*2**key_bound_exponent/2**(fft_bound_exponent)

def transform_hnp_instance_to_lattice_reduction_form(hnp_instance, prime):
    for i in range(len(hnp_instance)):
        hnp_instance[i][0] = (-1 * hnp_instance[i][0]) % prime

def nearest_plane_recursive(basis, GM_basis, x, dimension):
    dimension -= 1

    if dimension == 0:
        delta = round(x *  GM_basis.column(dimension))
        return delta * basis.column(dimension)

    gamma_d = x *  GM_basis.column(dimension)
    
    delta = round(gamma_d)
    
    v = delta * basis.column(dimension)
    x_prime = x - v
    
    y = nearest_plane_recursive(basis, GM_basis, x_prime, dimension)
    
    return y + v

def generate_n_bit_prime(n):
    lower_bound = 2^(n-1)
    upper_bound = 2^n - 1
    return random_prime(upper_bound, lbound=lower_bound)

def find_key_using_lattice_reduction(hnp_instance, prime):
    ring = Integers(prime)
    t_0 = ring(hnp_instance[0][0])
    t_0 = t_0.inverse()

    #eliminate the private key by substituting from 1st row
    hnp_instance_eliminated_key = []
    target = [0] * (len(hnp_instance) + 1)
    for i in range(len(hnp_instance)-1):
        c = hnp_instance[i+1][0] * t_0
        h = hnp_instance[i+1][1] - hnp_instance[i+1][0] * t_0 * hnp_instance[0][1]
        hnp_instance_eliminated_key.append([c, h])


    #build kannan embedding
    basis = matrix(len(hnp_instance_eliminated_key) + 2)
    basis[len(hnp_instance_eliminated_key), len(hnp_instance_eliminated_key)] = 1
    basis[len(hnp_instance_eliminated_key) + 1, len(hnp_instance_eliminated_key) + 1] = 1   
    for i in range(len(hnp_instance_eliminated_key)):
        basis[i,i] = prime
        basis[len(hnp_instance_eliminated_key), i] = hnp_instance_eliminated_key[i][0] 
        basis[len(hnp_instance_eliminated_key) + 1, i] = hnp_instance_eliminated_key[i][1]

    reduced_basis = basis.BKZ()

    k_0 = reduced_basis.row(0)[len(hnp_instance_eliminated_key)]
    return t_0*(hnp_instance[0][1]-k_0)



In [ ]:
#Range reduction

import math

LEAK_SIZE = 200
NUMBER_OF_SIGNATURES = 5000
MAX_FFT_SIZE_EXPONENT = 20 #maximalni velikost fft

log_prime = 256

prime = 60950016866452813043122729522080162838531715926523666195134120496553066206321  
key = 60000000000000000000000000000000000000000000000000000000000000000000000000000
hnp_instance = generate_hnp_instance(LEAK_SIZE, NUMBER_OF_SIGNATURES, prime, key) 



reduced_hnp_instance = []

d = 128
W = 2**(MAX_FFT_SIZE_EXPONENT-1) #weight for coefficients
G1 = 200 #1-norm coefficient bound
G_infty = 7 #infinity norm coefficient bound

for j in range(1, NUMBER_OF_SIGNATURES//d):

    #j = 1

    hnp_subset = hnp_instance[(j-1)*d:j*d]
    #build lattice
    basis = matrix(d+1)
    for i in range(d):
        basis[i,i] = W
        basis[i, d] = hnp_subset[i][0]

    basis[d,d] = prime

    reduced_basis = basis.BKZ()
    for i in range(d+1):
        row =reduced_basis.row(i)[:-1] / W
        if reduced_basis[i, d] < 2**MAX_FFT_SIZE_EXPONENT:
            #print("infty-norm", row.norm(infinity))
            if row.norm(infinity) < G_infty:
                #print("1-norm:", row.norm(infinity))
                if row.norm(1) < G1:
                    c = reduced_basis[i, d]
                    h = 0
                    for k in range(d):
                        h += row[k]*hnp_subset[k][1]
                    reduced_hnp_instance.append([c,h])


bias = []
for i in range(len(reduced_hnp_instance)):
    bias.append(abs((reduced_hnp_instance[i][0]*key - reduced_hnp_instance[i][1])%prime - prime))
print("redukovany bias je:", max(bias))





redukovany bias je: 60837152914065568535436588880507388562355916752175985135520665343326840459677


leak | # of signatures | pst po 30 iteracich
7    | 100             | 1
6    | 100             | 0.88
5    | 200             | 0.13
5    | 100             | 0.1
4    | 200             | 0

In [92]:
LEAK_SIZE = 4
NUMBER_OF_SIGNATURES =  100

bylo_to_spravne = 0.0
iterace = 10

for i in range(iterace):
    prime = generate_n_bit_prime(256) 
    key = randrange(2, prime-1)

    hnp_instance = generate_hnp_instance(LEAK_SIZE, NUMBER_OF_SIGNATURES, prime, key)

    transform_hnp_instance_to_lattice_reduction_form(reduced_hnp_instance, prime)

    asd = find_key_using_lattice_reduction(hnp_instance, prime)
    if asd == key:
        bylo_to_spravne += 1

print("pst je ", bylo_to_spravne/iterace)



pst je  0.000000000000000


In [ ]:
import math

LEAK_SIZE = 2
NUMBER_OF_SIGNATURES = 5000
MAX_FFT_SIZE_EXPONENT = 20 #maximalni velikost fft

log_prime = 256

prime = 60950016866452813043122729522080162838531715926523666195134120496553066206321  
key = 60000000000000000000000000000000000000000000000000000000000000000000000000000

hnp_instance = generate_hnp_instance_bounded(LEAK_SIZE, NUMBER_OF_SIGNATURES, prime, key, MAX_FFT_SIZE_EXPONENT)

#hnp_instance = sort_and_difference(hnp_instance, prime, 2, NUMBER_OF_RECOVERED_MSB)

cislo = bleichenbacher(hnp_instance, prime, MAX_FFT_SIZE_EXPONENT)
cislo2 = -round(cislo)%prime
msb_of_key = cislo2 - (cislo2 % 2**(log_prime-MAX_FFT_SIZE_EXPONENT)) #extract msbs  


print(msb_of_key)
print(bin(key))
print(bin(int(round(msb_of_key))))

59999917761042958852236804003353584149356568783024925294112094588145751818240
0b1000010010100110110010111110101001101001100101101000001000111110001011000101110000111100000001110111001001011110111110001101100011001100110000011110001110101000101010010111111001100000000000000000000000000000000000000000000000000000000000000000000000000000
0b1000010010100110110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [ ]:
#???????????????????????????????
import math

LEAK_SIZE = 2
NUMBER_OF_SIGNATURES = 1000000
MAX_FFT_SIZE_EXPONENT = 20


log_prime = 64

prime = 15559450162830958661  
key = 10000000000000000000

hnp_instance = generate_hnp_instance_bounded(LEAK_SIZE, NUMBER_OF_SIGNATURES, prime, key, MAX_FFT_SIZE_EXPONENT)

#hnp_instance = sort_and_difference(hnp_instance, prime, 2, MAX_FFT_SIZE)

solution = 0

for i in range(log_prime-MAX_FFT_SIZE_EXPONENT, 0, -MAX_FFT_SIZE_EXPONENT):
    cislo = 0
    msb_of_key = 0
    if(i==log_prime-MAX_FFT_SIZE_EXPONENT):
        cislo = bleichenbacher(hnp_instance, prime, MAX_FFT_SIZE_EXPONENT)
    else:
        cislo = bleichenbacher_bounded_key(hnp_instance, prime, MAX_FFT_SIZE_EXPONENT, i+MAX_FFT_SIZE_EXPONENT)
    cislo2 = -round(cislo)%prime
    
    msb_of_key = cislo2 - (cislo2 % 2**(i)) #extract msbs  
    solution += msb_of_key

    for j in range(len(hnp_instance)):
        hnp_instance[j][1] += hnp_instance[j][0]*msb_of_key % prime


print(solution)
print("the original key is: ", bin(key))
print("the difference is:", bin(msb_of_key))




0
the original key is:  0b1000101011000111001000110000010010001001111010000000000000000000
the difference is: 0b0
